In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import os

### Extracting Data From NHTSA GES Repository Years 2010-2015

Differing data files and formats across years, as well as storage directories made it difficult to obtain data using a batch iteration directly from the file transfer protocol set up by NHTSA. Directories for specific years were downloaded manually. Due to complex variation of feature column names, value categories and filenames, the attribute names, years and filepaths were first stored in data containers for ease in building a data cleaning script.  

In [ ]:
# directories storing all sas files needed for each year in NHTSA GES dataset
dir_years = [r"C:\Users\murra667\Documents\Springboard\Capstone_2\After 2010\GES10_PCSAS\repost GES", 
             r"C:\Users\murra667\Documents\Springboard\Capstone_2\After 2010\GES11_PCSAS", 
             r"C:\Users\murra667\Documents\Springboard\Capstone_2\After 2010\GES12_PCSAS", 
             r"C:\Users\murra667\Documents\Springboard\Capstone_2\After 2010\GES13_PCSAS", 
             r"C:\Users\murra667\Documents\Springboard\Capstone_2\After 2010\GES2014SAS", 
             r"C:\Users\murra667\Documents\Springboard\Capstone_2\After 2010\GES2015sas"]
# store dictionary of attributes to extract from each sas file type: person, accident, vehicle, vevent
sas_file_dict = {
              "person": 
              {"cols" :['VEHNO', 'VEH_NO', 'CASENUM', 'PER_TYP', 'AGE', 'SEX', 'ALC_RES', 'ALTRSULT']}, 
             "accident": 
              {"cols": ['CASENUM', 'MONTH', 'DAY_WEEK', 'YEAR', 'HOUR', 'MINUTE', 'LAND_USE',
                        'RELJCT2', 'REL_ROAD', 'WRK_ZONE',
              'INT_HWY', 'TYP_INT', 'NON_INVL', 'MAN_COL',
              'MAX_SEV', 'MAN_COLL', 'PERNOTMVIT']}, 
             "vehicle" :
              {"cols" : ['VEHNO', 'VEH_NO', 'CASENUM', 'TRAV_SP', 'VSPD_LIM', 'ACC_TYPE', 'MAKE', 'MODEL',
               'MOD_YEAR', 'MODEL_YR', 'DR_ZIP', 'VNUM_LAN', 'NUMOCCS', 'DZIPCODE']},
             "vevent": {"cols" : ['CASENUM', 'AOI1', 'GAD', 'VEHNO', 'VEH_NO']}
                }

# range of years stored in list 
years = list(range(2010,2016))
# store data first by file type (person, accident, vehicle, vevent) then by year 
for file in sas_file_dict.keys():
    sas_file_dict[file]['years'] = (dict(zip(years, dir_years)))

### First create a dataframe with Distracted Driving Cases

In [ ]:
# store dataframes of sas files containing distracted driving data
distracted_dfs = []
# iterate through years and their respective file directories
for year, file in zip(years, dir_years):
    global df
    df = pd.read_sas(os.path.join(file, "distract.sas7bdat"))
    if year == 2010:
        df = df.rename(columns = {"VEHNO": "VEH_NO"})
    df["VEH_NO"] = df["VEH_NO"].astype(str)
    df["CASENUM"] = df["CASENUM"].astype(str)
    # create unique identifier that creates a row for each car involved in
    # any crash
    df["CASENUM_VEH_NO"] = df["CASENUM"] + df["VEH_NO"]
    df = df.drop_duplicates(subset = "CASENUM_VEH_NO", keep = "first")
    distracted_dfs.append(df)

# concatenate the dataframes across years
global distracted_df
distracted_df = pd.concat(distracted_dfs)
distracted_dfs.clear()
distracted_df = distracted_df.drop_duplicates(keep = "first")

### Bring together NHTSA GES crash data across years and file types

In [ ]:
# update current_df during iteration
global current_df
# concat years of each file type (person, accident, event, )
concat_file_type = []
# iterate through filetype and respective columns in sas_file_dict
for file_type, file_type_attrs in sas_file_dict.items():
    # iterate through each year 
    for year, file_list in (file_type_attrs['years'].items()):
        filepath = os.path.join(file_list, (file_type + ".sas7bdat"))
        # access columns specific to year and file type
        columns = file_type_attrs['cols']
        # set mutable variable to adjust columns as necessary for year and file type
        global columns_adj
        # make a copy of columns 
        columns_adj = (columns.copy())
        # adjust columns as necessary for year and file type
        if year == 2010 and file_type != "accident": 
            columns_adj.remove("VEH_NO")
        if year ==  2010 and file_type == "accident":
            columns_adj.remove('MAN_COLL')
            columns_adj.remove('PERNOTMVIT')
        if year ==  2010 and file_type == "vevent":
            columns_adj.remove("AOI1")
        if year ==  2010 and file_type == "person":
            columns_adj.remove('ALC_RES')
        if year ==  2010 and file_type == "vehicle":
            columns_adj.remove("MOD_YEAR")
            columns_adj.remove("DR_ZIP")
        if year !=  2010 and file_type == "accident":
            columns_adj.remove('MAN_COL')
            columns_adj.remove('NON_INVL')
        if year !=  2010 and file_type != "accident":
            columns_adj.remove("VEHNO")
        if year !=  2010 and file_type == "person":
            columns_adj.remove('ALTRSULT')
        if year !=  2010 and file_type == "vehicle":
            columns_adj.remove("MODEL_YR")
            columns_adj.remove("DZIPCODE")
        if year !=  2010 and file_type == "vevent":
            columns_adj.remove("GAD")
        
        # read in current df and set columns appropriately 
        current_df = pd.read_sas(filepath)
        if file_type == "person":
            current_df = current_df.loc[current_df['PER_TYP'] == 1]
        current_df = current_df[columns_adj]
        
        # rename column names appropriately for year and file type for joining
        if year ==  2010 and file_type != "accident":
            current_df = current_df.rename(columns = {"VEHNO" : "VEH_NO"})
        if year ==  2010 and file_type == "accident": 
            current_df = current_df.rename(columns = {"MAN_COL" : "MAN_COLL", "NON_INVL" : "PERNOTMVIT"})
        if file_type == "person":
            current_df = current_df.rename(columns = {"ALC_RES" : "ALTRSULT"})
        if year ==  2010 and file_type == "vehicle":
            current_df = current_df.rename(columns = {"MODEL_YR" : "MOD_YEAR", "DZIPCODE" : "DR_ZIP"})
        if year ==  2010 and file_type == "vevent":
            current_df = current_df.rename(columns = {"GAD" : "AOI1"})
        current_df["CASENUM"] = current_df["CASENUM"].astype(str)
        if file_type != "accident":
            current_df = current_df.dropna(subset = ["CASENUM", "VEH_NO"]) 
            current_df["VEH_NO"] = current_df["VEH_NO"].astype(str)
            current_df["CASENUM_VEH_NO"] = current_df["CASENUM"] + current_df["VEH_NO"]
        else:
            current_df = current_df.dropna(subset = ["CASENUM"])
        #  for each file type, across years, add to container for concatenation
        concat_file_type.append(current_df)
        
    # concatenate file type across each year
    current_df = pd.concat(concat_file_type)
    # delete data conatiner after concatenation
    concat_file_type.clear()
    # use a special way to join the accident file, which has no 
    # vehicle number, and therefore no attribute CASENUM_VEH_NO which was
    # created for all other file types 
    if file_type == "accident":
        distracted_df = distracted_df.drop(columns = ["CASENUM_y"])
        distracted_df = pd.merge(left=distracted_df, right=current_df, left_on='CASENUM_x', right_on='CASENUM')
    # merge each file type into single dataframe (already concatenated by year)
    else:
        distracted_df = pd.merge(left = distracted_df, right = current_df, left_on="CASENUM_VEH_NO", right_on="CASENUM_VEH_NO")
     

### Perform final operations to save data out

In [ ]:
# drop duplicates along the CASENUM_VEH_NO column
final_df = distracted_df.drop_duplicates(subset = "CASENUM_VEH_NO")
# drop columns
final_df = final_df.drop(columns = ["CASENUM_x", "CASENUM_y", "VEH_NO_x", "VEH_NO_y", "PSUSTRAT", "PSU", "STRATUM", "PERNOTMVIT"])
# rename columns 
final_df = final_df.rename( columns = {"MDRDSTRD" : "distraction", "PJ" : "police jurisdiction", "REGION" : "region", "MODEL" : "model", "MOD_YEAR" : 'model year',
        "PER_TYP" : "person involved", "AGE" : "age", "SEX" : "sex", "ALTRSULT" : "alcohol", "DAY_WEEK" : "dow", "HOUR" : "hr", "INT_HWY" : "interstate hwy", "LAND_USE" : "population class", "MAN_COLL" : "collision",
        "MAX_SEV" : "injury", "MINUTE" : "min", "MONTH" : "month", "RELJCT2" : "specific location", "REL_ROAD" : "relation to road", "TYP_INT" : "intersection type", "WRK_ZONE" : "work zone", "VNUM_LAN" : "lanes", "AOI1" : "area of impact",
        "WEIGHT" : "weight", "TRAV_SP" : "speed", "VSPD_LIM" : "speed limit", "YEAR": "year", "ACC_TYPE" : "crash type", "DR_ZIP" : "zipcode", "MAKE": "make", "NUMOCCS" : "vehicle occupants"})

### Dataframe presents no need for dropping null items

In [ ]:
final_df.info()

In [ ]:
final_df.to_csv(r"C:\Users\murra667\Documents\Springboard\Capstone_2\After 2010\2010_2015_GES_distracted.csv")

In [22]:
final_df = pd.read_csv(r'C:\Users\murra667\Documents\Springboard\Capstone_2\After 2010\2010_2015_GES_distracted.csv')

C:\Users\murra667\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
final_df["distracted"] = 0

In [24]:
final_df.loc[(final_df['distraction'] != 0.0) | (final_df['distraction'] == 96.0) | (final_df['distraction'] == 99.0), 'distracted'] = 1

In [25]:
distracted = final_df.loc[final_df['distracted'] == 1]
non_distracted = final_df.loc[final_df['distracted'] == 0]

In [26]:
print ("Distracted Cases {}\nNon-Distracted Cases {}".format(len(distracted), len(non_distracted)))

Distracted Cases 98028
Non-Distracted Cases 437992
